# Introduction
This notebook contains a basic trend-following breakout strategy that can be used as a starting point to build and test your own strategy upon.

In [ ]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

Below is the barebone minumum to run a back test. It uses a feed and strategy that come out of the box with roboquant:

- A feed containing the prices for the 25 most active US stocks from the past 4 years (S&P 25)
- The EMA Crossover strategy with its default settings.

In [ ]:
val feed = AvroFeed.sp25()
val strategy = EMACrossover()
run(feed, strategy)

Typically however you'll want to develop your own strategy. Below a simple trend following strategy that implements the following rules:

- If the last price is the maximum price over the past <period> days, generate a BUY rating (1.0)
- If the last price is the minimum price over the past <period> days, generate a SELL rating (-1.0)
- In all other cases don't generate a rating

In [ ]:
/**
 * Basic breakout strategy.
 *
 * @param period the historic period to use for checking this is a breakout
 */
class MyStrategy(period: Int) : HistoricPriceStrategy(period) {

    /**
     * This method will be called by the superclass for every asset in the feed once
     * there is enough data collected for that asset.
     */
    override fun generateRating(data: DoubleArray): Double? {
        return when (data.last()) {
            data.max() -> BUY
            data.min() -> SELL
            else -> null
        }
    }

}

In [ ]:
// Create an instance of MyStrategy with 26 days worth of history
val strategy = MyStrategy(period = 26)

// Define the metric(s) you want to monitor
val journal = MemoryJournal(AccountMetric())

In [ ]:
// Run the back test
run(feed, strategy, journal)

In [ ]:
// Plot the equity curve
val equity = journal.getMetric("account.equity")
TimeSeriesChart(equity)

A very simple walkforward test over 1-year periods and at the end of the run printing the total equity hold in the account

In [ ]:
feed.timeframe.split(1.years).forEach { 
    val account = run(feed, EMACrossover(), timeframe=it)
    println("$it => ${account.equity()}")
}